<a href="https://colab.research.google.com/github/maxzameer/Brainstorm/blob/main/deep%20learning/cat_vs_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset


 98% 773M/788M [00:05<00:00, 148MB/s]
100% 788M/788M [00:05<00:00, 141MB/s]


In [3]:
import zipfile

In [4]:
zip_ref = zipfile.ZipFile('/content/microsoft-catsvsdogs-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()